In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from scipy.sparse import hstack
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
data = pd.read_csv('augmented_sents.txt', sep='\t', names=['context', 'question', 'answer'])

In [4]:
data.dropna(inplace=True)

In [5]:
data

,context,question,answer
0,Витя и Катя взяли из пакетика 40 конфетки.,"Сколько взяла Катя, если Витя взял 34?",6
1,Витя и Катя взяли из пакетика 49 конфетки.,"Сколько взяла Катя, если Витя взял 40?",9
2,Витя и Катя взяли из пакетика 48 конфетки.,"Сколько взяла Катя, если Витя взял 33?",15
3,Витя и Катя взяли из пакетика 38 конфетки.,"Сколько взяла Катя, если Витя взял 18?",20
4,Витя и Катя взяли из пакетика 28 конфетки.,"Сколько взяла Катя, если Витя взял 10?",18
5,Витя и Катя взяли из пакетика 24 конфетки.,"Сколько взяла Катя, если Витя взял 12?",12
6,Витя и Катя взяли из пакетика 29 конфетки.,"Сколько взяла Катя, если Витя взял 17?",12
7,Витя и Катя взяли из пакетика 22 конфетки.,"Сколько взяла Катя, если Витя взял 18?",4
8,Витя и Катя взяли из пакетика 30 конфетки.,"Сколько взяла Катя, если Витя взял 19?",11
9,Витя и Катя взяли из пакетика 37 конфетки.,"Сколько взяла Катя, если Витя взял 23?",14


In [4]:
tfidf = TfidfVectorizer(max_features=10000)
X_c = tfidf.fit_transform(data['context'])
X_q = tfidf.transform(data['question'])

In [5]:
X = hstack([X_c, X_q])
X = X.tocsr()

In [6]:
y = data['answer'].values

In [121]:
X_svd = TruncatedSVD(200).fit_transform(X)

In [12]:
X_with_manual = X_with_manual.tocsr()

In [13]:
oof_preds = np.zeros(len(y))
kf = KFold(n_splits=20, shuffle=True)
mae_scores = []
accuracies = []

for train_index, test_index in kf.split(X_with_manual):
    
    X_train, X_test = X_with_manual[train_index], X_with_manual[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = DecisionTreeRegressor()
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    preds[preds<0] = 1
    oof_preds[test_index] = preds
    mae_scores.append(mean_absolute_error(y_test, preds))
    preds = np.round(preds)
    accuracies.append(accuracy_score(y_test, preds))
    
print(np.mean(mae_scores))
print(np.mean(accuracies))

267.9809147509579
0.1992983716475096


In [15]:
len(train_index)

2746

In [25]:
clf.feature_importances_.argsort()[::-1]

array([2138, 2144, 2150, ..., 1511, 1510, 1077])

In [28]:
X_with_manual

<2890x2156 sparse matrix of type '<class 'numpy.float64'>'
	with 65649 stored elements in Compressed Sparse Row format>

In [19]:
accuracy_score(y_test, preds)

0.16666666666666666

In [199]:
mae_scores

[49.39310344827586,
 59.64597701149426,
 130.97931034482758,
 276.62068965517244,
 867.0068965517241,
 324.36551724137934,
 27.413793103448278,
 45.62758620689655,
 635.0413793103448,
 713.3862068965517,
 103.28472222222223,
 198.58333333333334,
 815.0416666666666,
 177.97222222222223,
 33.74305555555556,
 26.444444444444443,
 189.90972222222223,
 70.26388888888889,
 45.15277777777778,
 702.0833333333334]

In [7]:
def get_sum(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if nums:
        return float(sum(nums))
    else:
        return 0

def get_product(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if nums:
        prod = 1
        for num in nums:
            prod *= num
        return float(prod)
    else:
        return 0    

def get_aminusb(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if len(nums) == 2:
        res = float(nums[0] - nums[1])
        if res:
            return res
        else:
            return 0.
    else:
        return 0.
    
def get_bminusa(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if len(nums) == 2:
        res = float(nums[1] - nums[0])
        if res:
            return res
        else:
            return 0.
    else:
        return 0.
    
def get_adevideb(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if len(nums) == 2:
        res = round(nums[0]/nums[1])
        if res:
            return res
        else:
            return 0.
    else:
        return 0.
    
def get_bdevidea(text):
    nums = [int(x) for x in re.findall('\d+', text)]
    if len(nums) == 2:
        res = round(nums[1]/nums[0])
        if res:
            return res
        else:
            return 0.
    else:
        return 0.
    

In [8]:
data['text'] = data['context'] + data['question']

In [9]:
sums = data.text.apply(get_sum).values.reshape(-1, 1)
product = data.text.apply(get_product).values.reshape(-1, 1)
aminusb = data.text.apply(get_aminusb).values.reshape(-1, 1)
bminusa = data.text.apply(get_bminusa).values.reshape(-1, 1)

adevideb = data.text.apply(get_adevideb).values.reshape(-1, 1)
bdevidea = data.text.apply(get_bdevidea).values.reshape(-1, 1)

In [10]:
sums_context = data.context.apply(get_sum).values.reshape(-1, 1)
product_context = data.context.apply(get_product).values.reshape(-1, 1)
aminusb_context = data.context.apply(get_aminusb).values.reshape(-1, 1)
bminusa_context = data.context.apply(get_bminusa).values.reshape(-1, 1)

adevideb_context = data.context.apply(get_adevideb).values.reshape(-1, 1)
bdevidea_context = data.context.apply(get_bdevidea).values.reshape(-1, 1)

sums_question = data.question.apply(get_sum).values.reshape(-1, 1)
product_question = data.question.apply(get_product).values.reshape(-1, 1)
aminusb_question = data.question.apply(get_aminusb).values.reshape(-1, 1)
bminusa_question = data.question.apply(get_bminusa).values.reshape(-1, 1)

adevideb_question = data.question.apply(get_adevideb).values.reshape(-1, 1)
bdevidea_question = data.question.apply(get_bdevidea).values.reshape(-1, 1)

In [11]:
X_with_manual = hstack([X, sums, aminusb, bminusa, 
                        product, adevideb, bdevidea,
                       sums_context, aminusb_context, bminusa_context, 
                        product_context, adevideb_context, bdevidea_context,
                       sums_question, aminusb_question, bminusa_question, 
                        product_question, adevideb_question, bdevidea_question])

In [200]:
data['preds'] = oof_preds

In [195]:
data[data['preds'] != data['answer']]

,context,question,answer,preds
0,Витя и Катя взяли из пакетика 40 конфетки.,"Сколько взяла Катя, если Витя взял 34?",6,85.540821
1,Витя и Катя взяли из пакетика 49 конфетки.,"Сколько взяла Катя, если Витя взял 40?",9,82.192466
2,Витя и Катя взяли из пакетика 48 конфетки.,"Сколько взяла Катя, если Витя взял 33?",15,82.268865
3,Витя и Катя взяли из пакетика 38 конфетки.,"Сколько взяла Катя, если Витя взял 18?",20,65.105119
4,Витя и Катя взяли из пакетика 28 конфетки.,"Сколько взяла Катя, если Витя взял 10?",18,75.367727
5,Витя и Катя взяли из пакетика 24 конфетки.,"Сколько взяла Катя, если Витя взял 12?",12,76.559576
6,Витя и Катя взяли из пакетика 29 конфетки.,"Сколько взяла Катя, если Витя взял 17?",12,73.014329
7,Витя и Катя взяли из пакетика 22 конфетки.,"Сколько взяла Катя, если Витя взял 18?",4,74.202144
8,Витя и Катя взяли из пакетика 30 конфетки.,"Сколько взяла Катя, если Витя взял 19?",11,65.105119
9,Витя и Катя взяли из пакетика 37 конфетки.,"Сколько взяла Катя, если Витя взял 23?",14,77.563036


In [201]:
data[['answer', 'preds']]

,answer,preds
0,6,6.0
1,9,10.0
2,15,14.0
3,20,20.0
4,18,18.0
5,12,12.0
6,12,12.0
7,4,4.0
8,11,11.0
9,14,12.0


In [61]:
data.context.apply(lambda x: re.findall('\d+ \w+', x))

0                 [40 конфетки]
1                 [49 конфетки]
2                 [48 конфетки]
3                 [38 конфетки]
4                 [28 конфетки]
5                 [24 конфетки]
6                 [29 конфетки]
7                 [22 конфетки]
8                 [30 конфетки]
9                 [37 конфетки]
10                [49 конфетки]
11                [45 конфетки]
12                [37 конфетки]
13                [41 конфетки]
14                [39 конфетки]
15                [42 конфетки]
16                [49 конфетки]
17                [47 конфетки]
18                [18 конфетки]
19                [21 конфетки]
20                [48 конфетки]
21                [29 конфетки]
22                [26 конфетки]
23                [40 конфетки]
24                [30 конфетки]
25                [35 конфетки]
26                [18 конфетки]
27                [24 конфетки]
28                [33 конфетки]
29                [40 конфетки]
                 ...           
3249    